## 📈 Predicting Premier League Final Positions Using Betting Odds & Simulation

**Competition:** English Premier League 2025/26  
**Purpose:** Estimate probabilities of final league positions using betting market information and simulation  
**Methods:** Odds-implied probabilities, Monte Carlo simulation, scenario analysis  
**Author:** [Victoria Friss de Kereki](https://www.linkedin.com/in/victoria-friss-de-kereki/)  

---

**Notebook first written:** `17/01/2026`  
**Last updated:** `17/01/2026`  

> This notebook develops a probabilistic framework to predict final Premier League final positions using betting odds as market-based expectations.
>
> Betting odds are transformed into implied probabilities and adjusted for bookmaker margin. These probabilities are then used to simulate the remainder of the season via Monte Carlo methods, generating distributions over final points totals and league positions.
>
> The analysis focuses on estimating the likelihood of key outcomes such as title wins, top-four finishes, relegation, and mid-table placements. Results are presented at team level with uncertainty intervals, and the framework can be extended to incorporate form, fixture difficulty, or alternative predictive inputs beyond betting markets.


In [4]:
import soccerdata as sd

[01/18/26 10:34:26] INFO     No custom team name replacements found. You can configure these in       ]8;id=546848;file://C:\Users\vicky\AppData\Roaming\Python\Python312\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=378091;file://C:\Users\vicky\AppData\Roaming\Python\Python312\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\vicky\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=179283;file://C:\Users\vicky\AppData\Roaming\Python\Python312\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=514067;file://C:\Users\vicky\AppData\Roaming\Python\Python312\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\vicky\soccerdata\config\league_dict.json.                                    

## 1. Premier League Final Standings (ESPN Scraping)
##### Using the ESPN scraper I built in my previous project.

In [7]:
import pandas as pd

year = 2025  # current Premier League season start year

url = f"https://www.espn.com/soccer/standings/_/league/ENG.1/season/{year}"
tables = pd.read_html(url)

teams_raw = tables[0]
stats = tables[1]

teams = pd.DataFrame()
teams["position"] = teams_raw.iloc[:, 0].str.extract(r"^(\d+)").astype(int)
teams["team"] = (
    teams_raw.iloc[:, 0]
    .str.replace(r"^\d+", "", regex=True)
    .str.replace(r"^[A-Z]{2,3}", "", regex=True)
    .str.strip()
)

stats.columns = ["gp", "w", "d", "l", "gf", "ga", "gd", "pts"]
stats = stats.apply(lambda c: c.astype(str)
                              .str.replace("+", "", regex=False)
                              .astype(int))

premierleague = pd.concat([teams, stats], axis=1)
# premierleague["season"] = f"{year}-{year+1}"

premierleague


,position,team,gp,w,d,l,gf,ga,gd,pts
0,1,Arsenal,22,15,5,2,40,14,26,50
1,2,Manchester City,22,13,4,5,45,21,24,43
2,3,Aston Villa,21,13,4,4,33,24,9,43
3,4,Liverpool,22,10,6,6,33,29,4,36
4,5,Manchester United,22,9,8,5,38,32,6,35
5,6,Chelsea,22,9,7,6,36,24,12,34
6,7,Brentford,22,10,3,9,35,30,5,33
7,8,Sunderland,22,8,9,5,23,23,0,33
8,9,Newcastle United,21,9,5,7,32,27,5,32
9,10,Fulham,22,9,4,9,30,31,-1,31


## 2. Get betting odds using API

In [24]:
from dotenv import load_dotenv
import os

# Load variables from API_KEY.env
load_dotenv("API_KEY.env")

API_KEY = os.getenv("ODDS_DATA_API_KEY")

if api_key is None:
    raise ValueError("API_KEY not found. Check API_KEY.env")

print("API key loaded successfully")

API key loaded successfully


In [26]:
import requests
import pandas as pd
from datetime import datetime

url = "https://api.the-odds-api.com/v4/sports/soccer_epl/odds"

params = {
    "apiKey": API_KEY,
    "regions": "uk",
    "markets": "h2h",
    "oddsFormat": "decimal",
    "dateFormat": "iso",
    "days": 365  # get all upcoming matches for the next year
}

response = requests.get(url, params=params)
response.raise_for_status()

odds_data = response.json()
print("Total upcoming matches:", len(odds_data))

Total upcoming matches: 23


In [27]:
import pandas as pd

def flatten_odds(data):
    rows = []

    for match in data:
        match_id = match["id"]
        home = match["home_team"]
        away = match["away_team"]
        time = match["commence_time"]

        for book in match["bookmakers"]:
            bookmaker = book["title"]

            # Find h2h market
            h2h = next((m for m in book["markets"] if m["key"] == "h2h"), None)
            if not h2h:
                continue

            outcomes = {o["name"]: o["price"] for o in h2h["outcomes"]}

            rows.append({
                "match_id": match_id,
                "commence_time": time,
                "home_team": home,
                "away_team": away,
                "bookmaker": bookmaker,
                "home_odds": outcomes.get(home),
                "draw_odds": outcomes.get("Draw"),
                "away_odds": outcomes.get(away),
            })

    return pd.DataFrame(rows)

df = flatten_odds(odds_data)
df.head()

,match_id,commence_time,home_team,away_team,bookmaker,home_odds,draw_odds,away_odds
0,e15eb2362921b16a6b6a0397ce607a11,2026-01-18T14:00:00Z,Wolverhampton Wanderers,Newcastle United,Smarkets,5.90,3.40,1.81
1,e15eb2362921b16a6b6a0397ce607a11,2026-01-18T14:00:00Z,Wolverhampton Wanderers,Newcastle United,Paddy Power,5.00,3.25,1.73
2,e15eb2362921b16a6b6a0397ce607a11,2026-01-18T14:00:00Z,Wolverhampton Wanderers,Newcastle United,Sky Bet,5.00,3.30,1.75
3,e15eb2362921b16a6b6a0397ce607a11,2026-01-18T14:00:00Z,Wolverhampton Wanderers,Newcastle United,Betway,4.75,3.10,1.83
4,e15eb2362921b16a6b6a0397ce607a11,2026-01-18T14:00:00Z,Wolverhampton Wanderers,Newcastle United,888sport,4.60,3.25,1.75


In [28]:
avg_df = (
    df.groupby(["match_id", "home_team", "away_team"])
      .agg({
          "home_odds": "mean",
          "draw_odds": "mean",
          "away_odds": "mean"
      })
      .reset_index()
)

avg_df.head()

,match_id,home_team,away_team,home_odds,draw_odds,away_odds
0,1ca6d3d9cde3e58a39211feb9188530c,Newcastle United,Aston Villa,2.017647,3.608824,3.435294
1,1e811fa7ead0a3e6ef920b15b2bbb95d,Burnley,Tottenham Hotspur,3.802778,3.450000,1.965556
2,342788786c22e570ed2da53a9608113f,Brighton and Hove Albion,Bournemouth,1.855000,3.933333,3.808333
3,36820753efb36739a83c6e5e440827b2,Brighton and Hove Albion,Everton,1.800000,3.110000,3.496250
4,38a3cb5e295f55e274d589fc646cf2dd,Tottenham Hotspur,Manchester City,4.610000,3.581250,1.600000


## 3. Get fixtures.

In [29]:
# Load variables from API_KEY.env
load_dotenv("API_KEY.env")

API_KEY = os.getenv("FOOTBALL_DATA_API_KEY")

if api_key is None:
    raise ValueError("API_KEY not found. Check API_KEY.env")

print("API key loaded successfully")

API key loaded successfully


In [32]:
import requests
import pandas as pd
from datetime import datetime, timedelta

url = "https://api.football-data.org/v4/competitions/PL/matches"

headers = {
    "X-Auth-Token": API_KEY
}

today = datetime.utcnow().date()
end_of_season = today + timedelta(days=365)  # big range to cover all remaining games

params = {
    "status": "SCHEDULED",
    "dateFrom": today.isoformat(),
    "dateTo": end_of_season.isoformat()
}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()

data = response.json()
fixtures = data["matches"]

df_fixtures = pd.DataFrame(fixtures)

df_fixtures_clean = df_fixtures[[
    "utcDate",
    "status",
    "homeTeam",
    "awayTeam"
]]

df_fixtures_clean.head()
print("Total scheduled matches:", len(df_fixtures_clean))


[01/18/26 11:48:29] WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_30580\613972728.py:11:     ]8;id=413180;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=134468;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             DeprecationWarning: datetime.datetime.utcnow() is deprecated and                      
                             scheduled for removal in a future version. Use timezone-aware objects                 
                             to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).                   
                               today = datetime.utcnow().date()                                                    
                                                                                                                   

Total scheduled matches: 162


In [33]:
df_fixtures_clean["homeTeam"] = df_fixtures_clean["homeTeam"].apply(lambda x: x["name"])
df_fixtures_clean["awayTeam"] = df_fixtures_clean["awayTeam"].apply(lambda x: x["name"])

[01/18/26 11:48:42] WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_30580\2614041168.py:1:     ]8;id=342028;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=508086;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             SettingWithCopyWarning:                                                               
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_fixtures_clean["homeTeam"] =                                                     
                             df_fixtures_clean["homeTeam"].apply(lambda x: x["name"])                              
                                                                                                                   

                    WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_30580\2614041168.py:2:     ]8;id=695825;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=975843;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             SettingWithCopyWarning:                                                               
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_fixtures_clean["awayTeam"] =                                                     
                             df_fixtures_clean["awayTeam"].apply(lambda x: x["name"])                              
                                                                                                                   

In [34]:
df_fixtures_clean

,utcDate,status,homeTeam,awayTeam
0,2026-01-18T16:30:00Z,TIMED,Aston Villa FC,Everton FC
1,2026-01-19T20:00:00Z,TIMED,Brighton & Hove Albion FC,AFC Bournemouth
2,2026-01-24T12:30:00Z,TIMED,West Ham United FC,Sunderland AFC
3,2026-01-24T15:00:00Z,TIMED,Burnley FC,Tottenham Hotspur FC
4,2026-01-24T15:00:00Z,TIMED,Fulham FC,Brighton & Hove Albion FC
...,...,...,...,...
157,2026-05-24T15:00:00Z,TIMED,Liverpool FC,Brentford FC
158,2026-05-24T15:00:00Z,TIMED,Manchester City FC,Aston Villa FC
159,2026-05-24T15:00:00Z,TIMED,Nottingham Forest FC,AFC Bournemouth
160,2026-05-24T15:00:00Z,TIMED,Tottenham Hotspur FC,Everton FC


## Get this season results (2025/26)

In [35]:
url = "https://api.football-data.org/v4/competitions/PL/matches"
params = {
    "season": 2025,   # season year
    "status": "FINISHED"
}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()
past_matches = response.json()["matches"]

In [39]:
clean_rows = []

for m in past_matches:
    row = {
        "utcDate": m["utcDate"],
        "matchday": m["matchday"],
        "status": m["status"],
        "homeTeam": m["homeTeam"]["name"],
        "awayTeam": m["awayTeam"]["name"],
        "homeGoals": m["score"]["fullTime"]["home"],
        "awayGoals": m["score"]["fullTime"]["away"],
        "winner": m["score"]["winner"]
    }
    clean_rows.append(row)

past_matches_clean = pd.DataFrame(clean_rows)
past_matches_clean.head()

,utcDate,matchday,status,homeTeam,awayTeam,homeGoals,awayGoals,winner
0,2025-08-15T19:00:00Z,1,FINISHED,Liverpool FC,AFC Bournemouth,4,2,HOME_TEAM
1,2025-08-16T11:30:00Z,1,FINISHED,Aston Villa FC,Newcastle United FC,0,0,DRAW
2,2025-08-16T14:00:00Z,1,FINISHED,Brighton & Hove Albion FC,Fulham FC,1,1,DRAW
3,2025-08-16T14:00:00Z,1,FINISHED,Sunderland AFC,West Ham United FC,3,0,HOME_TEAM
4,2025-08-16T14:00:00Z,1,FINISHED,Tottenham Hotspur FC,Burnley FC,3,0,HOME_TEAM
